In [1]:
from langchain_community.chat_models import ChatOllama  # 使用 Ollama 封裝的 LLaMA 模型
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import PromptTemplate
import torch


# 使用本地 LLaMA 模型
llm_llama3_8B = ChatOllama(model="llama3:8B")
llm_phi3_3dot8B = ChatOllama(model="phi3:3.8B")
llm_mistral_7B = ChatOllama(model="mistral:7B")
llm_gemma3_4B = ChatOllama(model="gemma3:4B")

# GPU 加速
print("=== PyTorch GPU 加速環境檢查 ===")
print(f"PyTorch 版本: {torch.__version__}")
print(f"PyTorch 編譯的 CUDA 版本: {torch.version.cuda}")
print(f"是否支援 CUDA: {torch.cuda.is_available()}")

try:
    if torch.cuda.is_available():
        gpu_count = torch.cuda.device_count()
        current_gpu = torch.cuda.current_device()
        device_name = torch.cuda.get_device_name(current_gpu)

        print(f"偵測到 {gpu_count} 個 GPU")
        print(f"當前使用的 GPU：{device_name}")
        device = torch.device("cuda")
    else:
        raise RuntimeError("CUDA 不可用，將使用 CPU")
except Exception as e:
    print(f"無法使用 GPU：{e}")
    device = torch.device("cpu")

print(f"pytorch可用的GPU為：{device}")


c:\Users\TUF\Documents\base-agnet\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


=== PyTorch GPU 加速環境檢查 ===
PyTorch 版本: 2.8.0+cpu
PyTorch 編譯的 CUDA 版本: None
是否支援 CUDA: False
無法使用 GPU：CUDA 不可用，將使用 CPU
pytorch可用的GPU為：cpu


C:\Users\TUF\AppData\Local\Temp\ipykernel_4840\3167653020.py:10: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm_llama3_8B = ChatOllama(model="llama3:8B")


In [2]:
%load_ext autoreload
%autoreload 2

### 建立工具 ###

In [3]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from memory import Memory, MemoryStream
from reflection import ReflectionEngine

# 所有 Agent 共用同一個 MemoryStream，彼此回覆會被寫回 FAISS，後續查詢時能互相參考。
# === 1. 建立共享記憶流 ===
memory_stream = MemoryStream()

# === 2. 封裝記憶檢索工具 ===
memory_tool = Tool(
    name="MemoryRetriever",
    func=lambda q: "\n".join(m.content for m in memory_stream.search_by_vector(q)),
    description="檢索過往的重要對話與事件"
)

# initialize_agent 的 tools 參數必須是 List[Tool]
tools = [memory_tool]


c:\Users\TUF\Documents\base-agnet\memory.py:24: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
c:\Users\TUF\Documents\base-agnet\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 建立 Agent ###

In [4]:
# === 3. 定義多個角色與對應 LLM ===
# 新增角色，加入新的 (角色描述, llm) 配對
roles = {
    "llama":  ("家醫科醫生", llm_llama3_8B),
    "mistral":("內科醫生", llm_mistral_7B),
    "gemma":  ("外科醫生", llm_gemma3_4B)
}

agents = {}
for name, (role_desc, llm_model) in roles.items():
    prompt_prefix = f"你是一位{role_desc}，回答時請保持此角色的語氣與思考方式。"
    
    agents[name] = initialize_agent(
        tools=tools,
        llm=llm_model,
        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        verbose=True,
        agent_kwargs={
            "prefix": prompt_prefix,
            }
    )


C:\Users\TUF\AppData\Local\Temp\ipykernel_4840\1564673074.py:13: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agents[name] = initialize_agent(


In [5]:
# === 4. 對話流程示例 ===
def multi_agent_round(user_input: str):
    print(f"\n=== 使用者輸入 ===\n{user_input}\n")
    # 對每個 agent 輪詢
    for name, agent in agents.items():
        response = agent.run({
            "input": user_input,
            "chat_history": []
        })
        print(f"[{name}] {response}\n")
        # 將回覆寫入共享記憶
        memory_stream.add_memory(
            f"{name} 回覆: {response}",
            importance=5
        )

def diagnose_round(medical_record: dict):
    patient_name = medical_record["patient_name"]
    notes = medical_record["notes"]

    print(f"\n=== 病歷分析 ===")
    print(f"病人姓名：{patient_name}")
    print(f"主訴與觀察紀錄：{notes}\n")

    diagnoses = {}

    # === 各專科醫師（agent）給出初步意見 ===
    for name, agent in agents.items():
        role = roles[name][0]
        prompt = f"""
        病人資料如下：
        姓名：{patient_name}
        病徵描述：{notes}

        請以一位{role}的身份，
        說明此病人可能的疾病原因，並建議該掛哪一科。
        """
        response = agent.run({
            "input": prompt,
            "chat_history": []
        })
        print(f"[{role}] 的診斷：{response}\n")

        diagnoses[name] = response

        # 寫入記憶
        memory_stream.add_memory(
            f"{role} 判斷：{response}",
            importance=5
        )

    # === 整合階段 ===
    integration_prompt = f"""
    以下是三位專科醫師的診斷意見：
    {diagnoses}

    請你根據這些診斷，
    給出最合理的「最終建議科別」，並簡述理由（請用繁體中文）。
    回答格式：
    建議科別：XXX
    理由：...
    """

    final_decision = llm_llama3_8B.invoke(integration_prompt)
    print(f"\n=== 最終判斷結果 ===\n{final_decision.content}\n")

    # 寫入記憶
    memory_stream.add_memory(
        f"最終判斷：{final_decision.content}",
        importance=8
    )



In [6]:
# === 5. 測試對話 ===
# 如果是 .py 就要加判斷式
# if __name__ == "__main__":
#     multi_agent_round("請大家一起討論氣候變遷對農業的影響。")
#     multi_agent_round("根據剛剛的討論，再給我一段結論。")
medical_records = [
    {"patient_name": "陳怡君", "notes": "病人右前臂出現紅疹並有輕微滲液"},
    {"patient_name": "王建國", "notes": "左膝關節疼痛兩週，走路時加劇"},
    {"patient_name": "李芳", "notes": "喉嚨刺痛並伴隨輕微發燒"},
]


# multi_agent_round("請大家一起討論氣候變遷對農業的影響。")
# multi_agent_round("根據剛剛的討論，再給我一段結論。")

for record in medical_records:
        diagnose_round(record)


=== 病歷分析 ===
病人姓名：陳怡君
主訴與觀察紀錄：病人右前臂出現紅疹並有輕微滲液



> Entering new AgentExecutor chain...


C:\Users\TUF\AppData\Local\Temp\ipykernel_4840\2394925543.py:38: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run({


```json
{
    "action": "MemoryRetriever",
    "action_input": "Search for diseases related to red rash on the right forearm"
}
```

Please use MemoryRetriever to look up information and provide possible diseases and recommendations.
Observation: 
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Based on the patient's symptoms, it is possible that they may be experiencing a skin manifestation related to underlying conditions such as eczema, dermatitis, or even sepsis. Further evaluation and testing would be necessary to determine the exact cause. I would recommend consulting with a dermatologist for further examination and treatment."
}

> Finished chain.
[家醫科醫生] 的診斷：Based on the patient's symptoms, it is possible that they may be experiencing a skin manifestation related to underlying conditions such as eczema, dermatitis, or even sepsis. Further evaluation and testing would be necessary to determine the exact cause. I would recommend consulting with a dermatologis

In [7]:
# === 6. 反思與長期記憶整理 ===
reflector = ReflectionEngine(llm=llm_llama3_8B,
                             memory_stream=memory_stream,
                             reflection_threshold=20)
reflector.reflect()

已完成反思，新增記憶：
 - Here are three high-level insights in natural Chinese:
 - 「皮膚科是最合理的選擇」，皮膚疾病或感染可能性高。
 - 「Orthopedic Specialist 是可能的選擇」，關節相關症狀可能存在。
 - 「ENT專家是最合理的下一步」，疝痛和發熱可能與 ENT 相關疾病有關。


['Here are three high-level insights in natural Chinese:',
 '「皮膚科是最合理的選擇」，皮膚疾病或感染可能性高。',
 '「Orthopedic Specialist 是可能的選擇」，關節相關症狀可能存在。',
 '「ENT專家是最合理的下一步」，疝痛和發熱可能與 ENT 相關疾病有關。']

In [8]:
from conversation_loop import ConversationLoop

# 建立對話迴圈物件
conv_loop = ConversationLoop(
    agents=agents,
    memory_stream=memory_stream,
    reflector=reflector,
    reflection_interval=2   # 每兩輪觸發一次反思
)

# 啟動對話流程
# conv_loop.run_round("請大家一起討論氣候變遷對農業的影響。")
# conv_loop.run_round("根據剛剛的討論，再給我一段結論。")
# conv_loop.run_round("請提出可行的調適策略。")  # 這一輪會觸發反思

conv_loop.run_medical_case(medical_records)


=== 病歷討論開始：陳怡君 ===
病歷摘要：病人右前臂出現紅疹並有輕微滲液



> Entering new AgentExecutor chain...
Here is my response:

```json
{
    "action": "MemoryRetriever",
    "action_input": "dermatology"
}
```

I would like to use the MemoryRetriever tool to look up information about dermatology and discuss which medical department should be referred to based on the patient's symptoms, including red rashes and mild swelling.
Observation: 家醫科醫生 判斷：Based on the patient's symptoms, it is possible that they may be experiencing a skin manifestation related to underlying conditions such as eczema, dermatitis, or even sepsis. Further evaluation and testing would be necessary to determine the exact cause. I would recommend consulting with a dermatologist for further examination and treatment.
外科醫生 判斷：Considering the patient’s presentation of red, weeping rash on the anterior arm, the most likely possibilities include eczema, contact dermatitis, or, less likely but requiring immediate consideration, infection. Given 